# Hallucination Detection

1. Extractor Keywords
   1. Remove Stop Words:
      1. Chinese (zh): jieba
      2. Arabic (ar): Hugging Face (asafaya/bert-base-arabic)
      3. Hindi (hi): indic-nlp-library
      4. Basque (eu): tokenize - xx_ent_wiki_sm, Snowball - stopwords-eu.txt
      5. Czech (cs): stopwordsios
      6. Farsi (fa): Hazm
      7. Other Languages: spaCy model
   2. Recognize NER Entities:
      1. Hugging Face Models:
         1. Arabic (ar): asafaya/bert-base-arabic
         2. Catalan (ca): projecte-aina/roberta-base-ca-v2-cased-ner
         3. Farsi (fa): HooshvareLab/bert-fa-base-uncased-ner-arman
         4. Other Languages: FacebookAI/xlm-roberta-large-finetuned-conll03-english
      3. For Unrecognized Content, Perform Tokenization (Extract Key Nouns if Possible):
         1. Chinese (zh): jieba (tfidf-keywords)
         2. Hindi (hi): indic_tokenize
         3. Arabic (ar): Hugging Face (asafaya/bert-base-arabic)
         4. Czech (cs): Stanza
         5. Farsi (fa): Stanza
         6. Other Languages: spaCy tokenize

2. Acquire External Knowledge:
  1. Use Baidu Translate API to translate all extracted key phrases into English as a fallback mechanism for retrieval.
  2. Retrieval Rollback Mechanism:
First, use the key phrases in the target language to search via the Wikipedia API.
    1. If the search fails, use the translated English phrases for retrieval.
Note: During retrieval, there might be errors due to Traditional Chinese redirects. These need to be cleared, and results in Traditional Chinese should be forcefully converted.
  3. Extract the first 200 characters from the search results.
  
3. Use (model_input, model_output_text, context) to detect hallucination words and their probabilities via GPT-3.5.

4. Merge overlapping words and compute their probabilities using Exponentiation.

5. Create Soft Labels: Identify hallucination word positions in the model_output_text and combine them with their computed probabilities.

In [ ]:
from openai import OpenAI
import requests
import httpx
import json
import pandas as pd
from tqdm import tqdm
import ast
from scorer import recompute_hard_labels, load_jsonl_file_to_records, score_iou, score_cor, main
import numpy as np
from langdetect import detect, LangDetectException
import re
import os
import glob

In [ ]:
# set OpenAI API and proxies
api_key = ""

proxies = {
    "http": "http://127.0.0.1:10809",
    "https": "http://127.0.0.1:10809"
}

In [ ]:
prompt_template = """
You are an AI model output evaluation expert, responsible for detecting hallucinated words in model output and assigning accurate probability scores to each hallucination.

Below is the input information:
- **Language**: {language} (e.g., en(English), ar(Arabic), es(Spanish), etc.)
- **Question**: {question}
- **Model Output**: {output}
- **Background Knowledge** (if available): {context}

### **Task**:
Your task is to:
1. **Identify hallucinated words or phrases** in the model output based on the question and background knowledge.
   - A word or phrase is considered a hallucination if it:
     - Contradicts the background knowledge.
     - Is unverifiable or fabricated.
     - Contains logical inconsistencies.
2. **Assign a probability score** to each hallucinated word or phrase according to the following criteria:
   - **Probability > 0.7**: Severe factual errors or contradictions.
   - **Probability 0.5 - 0.7**: Unverifiable or speculative content.
   - **Probability 0.3 - 0.5**: Minor inconsistencies or unverifiable details.
   - **Probability 0.1 - 0.3**: Minor inaccuracies or vague ambiguities.
   - **Do not label words with probability ≤ 0.1** (i.e., verifiable facts).

### **Additional Instructions**:
- Do **not** mark redundant or overly generic words (e.g., "the", "a", "and") as hallucinations unless they introduce factual errors.
- Pay special attention to:
  - **Numerical data** (e.g., dates, quantities, percentages).
  - **Named entities** (e.g., people, organizations, locations).
  - **Logical contradictions** (e.g., self-contradictions within the text).
- If background knowledge is absent, base your judgment solely on internal consistency.

### **Example**:
#### Input:
- **Question**: "What year did Einstein win the Nobel Prize?"
- **Model Output**: "Einstein won the Nobel Prize in Physics in 1922 for his discovery of the photoelectric effect."
- **Background Knowledge**: "Einstein won the Nobel Prize in Physics in 1921."

#### Output:
[
    {{"word": "1922", "prob": 0.9}}
]

### **Output Format**:
Return the result as a JSON array:
[
    {{"word": <example_word>, "prob": <probability>}},
    {{"word": <another_word>, "prob": <probability>}}
]

### Important:
- Provide precise word-level annotations.
- Do not include any text or explanations outside the JSON array.
"""


In [ ]:
def evaluate_with_selfcheck(question, output, context="", language="en", n=5, retries=3):

    if context is None:
        context = ""

    language = language.lower()

    prompt = prompt_template.format(question=question, output=output, context=context, language=language)

    for attempt in range(retries):
        try:
            response = requests.post(
                "https://api.openai.com/v1/chat/completions",
                headers={
                    "Authorization": f"Bearer {api_key}",
                    "Content-Type": "application/json"
                },
                json={
                    "model": "gpt-3.5-turbo",
                    "messages": [{"role": "user", "content": prompt}],
                    "n": n
                },
                proxies=proxies
            )

            if response.status_code == 200:
                content = response.json()["choices"][0]["message"]["content"]
                try:
                    return json.loads(content)
                except json.JSONDecodeError as e:
                    print(f"Failed to parse JSON content: {content}. Error: {e}")
                    return []
            else:
                print(f"Request failed with status code: {response.status_code}")
                print(f"Response: {response.text}")
                return []

        except requests.exceptions.RequestException as e:
            print(f"Request failed: {e}")
            return []

    print("Retry limit exceeded, returning empty result")
    return []

In [ ]:
# Locate word positions in the original text
def locate_word_positions(words_with_probs, model_output_text):
    ranges = []
    for item in words_with_probs:
        word = item["word"]
        prob = item["prob"]
        start_idx = model_output_text.find(word)
        while start_idx != -1:
            end_idx = start_idx + len(word)
            ranges.append((start_idx, end_idx, prob))
            start_idx = model_output_text.find(word, end_idx)
    return ranges

# Merge overlapping ranges
def merge_ranges(ranges):
    if not ranges:
        return []
    # Sort ranges by start position
    ranges.sort(key=lambda x: x[0])
    merged = [ranges[0]]
    for current in ranges[1:]:
        last = merged[-1]
        if current[0] <= last[1]:  # Overlapping
            new_end = max(last[1], current[1])
            new_prob = (last[2] + current[2]) / 2  # Average probabilities
            merged[-1] = (last[0], new_end, new_prob)
        else:
            merged.append(current)
    return merged

# Compute average probabilities with enhanced overlap weighting
def compute_average_probability_v3(merged_ranges, all_ranges):
    avg_probs = []
    for m_start, m_end, _ in merged_ranges:
        total_prob = 0
        total_overlap_weight = 0

        for r_start, r_end, prob in all_ranges:
            # Calculate overlap length
            overlap_start = max(m_start, r_start)
            overlap_end = min(m_end, r_end)
            overlap_length = max(0, overlap_end - overlap_start)

            # Add weighted contribution (consider overlap frequency)
            if overlap_length > 0:
                weight = overlap_length  # Base weight is overlap length
                total_prob += prob * weight
                total_overlap_weight += weight

        # Adjust probability by total weight (with enhancement factor)
        if total_overlap_weight > 0:
            final_prob = (total_prob / total_overlap_weight) ** 1.2  # Enhancing frequent overlaps
        else:
            final_prob = 0  # No overlap, probability is zero

        avg_probs.append(final_prob)
    return avg_probs

# Main function to process hallucination detection
def process_hallucination_detection(question, model_output_text, context, language):
    # Call GPT model to get hallucinated words and probabilities
    hallucinations = evaluate_with_selfcheck(question, model_output_text, context, language)
    # print("Hallucinations detected:", hallucinations)

    # Filter out hallucinations with probability <= 0.1
    hallucinations = [item for item in hallucinations if item["prob"] > 0.1]

    # Locate hallucination positions in the model output text
    hallucination_ranges = locate_word_positions(hallucinations, model_output_text)
    # print("Hallucination Ranges:", hallucination_ranges)

    # Merge overlapping ranges
    merged_ranges = merge_ranges(hallucination_ranges)
    # print("Merged Ranges:", merged_ranges)

    # Compute final probabilities for merged ranges
    final_probabilities = compute_average_probability_v3(merged_ranges, hallucination_ranges)

    # Prepare final output
    result = []
    for i, (start, end, _) in enumerate(merged_ranges):
        result.append({
            "start": start,
            "end": end,
            "prob": final_probabilities[i]
        })
    return result

In [ ]:
def process_dataset(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    input_files = glob.glob(os.path.join(input_folder, "*.jsonl"))

    with tqdm(total=len(input_files), desc="Processing Files", unit="file") as file_progress:
        for file_path in input_files:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = [json.loads(line) for line in f]

            output_data = []

            with tqdm(total=len(data), desc=f"Processing {os.path.basename(file_path)}", unit="entry", leave=False) as entry_progress:
                for entry in data:
                    try:
                        question = entry.get("model_input", "")
                        model_output_text = entry.get("model_output_text", "")
                        context = entry.get("wikipedia_context", "")
                        language = entry.get("lang", "").lower()

                        soft_labels = process_hallucination_detection(
                            question, model_output_text, context, language
                        )
                        hard_labels = recompute_hard_labels(soft_labels)

                        output_entry = {
                            "id": entry.get("id"),
                            "lang": entry.get("lang"),
                            "model_input": entry.get("model_input"),
                            "model_output_text": entry.get("model_output_text"),
                            "model_id": entry.get("model_id"),
                            "soft_labels": soft_labels,
                            "hard_labels": hard_labels,
                            "model_output_logits": entry.get("model_output_logits"),
                            "model_output_tokens": entry.get("model_output_tokens")
                        }

                        output_data.append(output_entry)

                    except Exception as e:
                        print(f"Error processing entry {entry.get('id')}: {e}")

                    entry_progress.update(1)

            output_file = os.path.join(output_folder, os.path.basename(file_path))
            with open(output_file, 'w', encoding='utf-8') as f:
                for item in output_data:
                    f.write(json.dumps(item, ensure_ascii=False) + '\n')

            file_progress.update(1)
            print(f"Processed and saved: {output_file}")

In [ ]:
input_folder = "../data/detect_val/exknowledge_m2/"
output_folder = "../data/detect_val/detect_gpt_m2/"

process_dataset(input_folder, output_folder)

## Evaluation

In [ ]:
def evaluate_iou_and_cor(val_dir, detect_dir, output_file):
    """
    Evaluate IoU and Spearman correlation between the reference (val) and detected (detect) files.

    :param val_dir: Directory containing the ground truth files (e.g., data/val/val/)
    :param detect_dir: Directory containing the detected files (e.g., data/detect/)
    :param output_file: Path to save the evaluation results (optional)
    """
    # List all files in the validation directory
    val_files = os.listdir(val_dir)
    detect_files = os.listdir(detect_dir)

    # Ensure that we are comparing the same files (same lang)
    for val_file in val_files:
        # Skip non-JSONL files
        if not val_file.endswith('.jsonl'):
            continue

        # Remove the first 'val/' part from the file path to match the structure of detect directory
        detect_file_name = val_file.replace('val/', '')  # Remove 'val/' from the file name

        # Check if the corresponding detect file exists
        detect_file_path = os.path.join(detect_dir, detect_file_name)

        if not os.path.exists(detect_file_path):
            print(f"Warning: {detect_file_path} not found, skipping.")
            continue

        # Load ground truth (val) and detected (detect) data
        ref_dicts = load_jsonl_file_to_records(os.path.join(val_dir, val_file))
        pred_dicts = load_jsonl_file_to_records(detect_file_path)

        # Calculate IoU and Spearman correlation
#        try:
        ious, cors = main(ref_dicts, pred_dicts)
#        except IndexError as e:
 #           print(f"IndexError occurred for file: {val_file}, skipping this file. Error: {e}")
  #          continue

        # Print or save the results
        print(f"Results for {val_file}:")
        print(f"  Mean IoU: {ious.mean():.8f}")
        print(f"  Mean Spearman Correlation: {cors.mean():.8f}")

        # Optionally, save the results to a file
        if output_file:
            with open(output_file, 'a', encoding='utf-8') as f:
                f.write(f"Results for {val_file}:\n")
                f.write(f"  Mean IoU: {ious.mean():.8f}\n")
                f.write(f"  Mean Spearman Correlation: {cors.mean():.8f}\n\n")

val_dir = '../data/val/val/'
detect_dir = '../data/detect_val/detect_gpt_m2/'
output_file = 'evaluation_results.txt'
evaluate_iou_and_cor(val_dir, detect_dir, output_file)